# Data Preparation

Prepare train/val/test splits for coconut tree detection.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kshitijrajsharma/dl4cv-object-detection-on-aerial-imagery/blob/master/notebooks/experiment.ipynb)

In [ ]:
# !pip install dl4cv_oda

In [ ]:
import requests
import geopandas as gpd
from pathlib import Path
from dl4cv_oda import (clean_osm_data, clip_labels_to_tiles, convert_to_yolo_format,
                       create_train_val_split, create_yolo_config, download_tiles)

In [ ]:
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
CHIPS_DIR = DATA_DIR / "chips"
LABELS_DIR = DATA_DIR / "labels"
YOLO_DIR = DATA_DIR / "yolo"

OSM_FILE = RAW_DIR / "kolovai-trees.geojson"
CLEANED_FILE = RAW_DIR / "cleaned.geojson"
TREES_BOX_FILE = DATA_DIR / "trees_box.geojson"
TILES_FILE = DATA_DIR / "tiles.geojson"

if not OSM_FILE.exists():
    OSM_FILE.parent.mkdir(parents=True, exist_ok=True)
    OSM_FILE.write_bytes(requests.get("https://github.com/kshitijrajsharma/dl4cv-oda/blob/master/data/raw/kolovai-trees.geojson?raw=true", allow_redirects=True).content)
    print(f"Downloaded OSM data")

if not CLEANED_FILE.exists():
    count = clean_osm_data(str(OSM_FILE), str(CLEANED_FILE), str(TREES_BOX_FILE))
    print(f"Cleaned {count} trees")

if not TILES_FILE.exists():
    data = gpd.read_file(TREES_BOX_FILE)
    data.to_crs(epsg=4326, inplace=True)
    bbox = list(data.total_bounds)
    await download_tiles(bbox, 19, "https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/{z}/{x}/{y}.png", DATA_DIR, 'OAM')
    print("Downloaded tiles")

In [ ]:
if not (YOLO_DIR / "train").exists():
    stats = clip_labels_to_tiles(str(TREES_BOX_FILE), str(TILES_FILE), str(LABELS_DIR))
    print(f"Clipped {stats['total_trees']} trees to {stats['processed']} tiles")
    
    class_mapping = convert_to_yolo_format(str(TREES_BOX_FILE), str(CHIPS_DIR), str(LABELS_DIR), str(YOLO_DIR), target_species="Coconut")
    print(f"Converted to YOLO format")
    
    train_count, val_count, test_count = create_train_val_split(str(LABELS_DIR), str(CHIPS_DIR), str(YOLO_DIR), train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42)
    print(f"Split: train={train_count}, val={val_count}, test={test_count}")
    
    config_file = create_yolo_config(str(YOLO_DIR), {"Coconut": 0})
    print(f"Config: {config_file}")

print("Data preparation complete")

# Model Training

In [ ]:
import json
from datetime import datetime
import pandas as pd
from ultralytics import YOLO, RTDETR
import torch

## Configuration

In [ ]:
RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)

SEED = 42
IMG_SIZE = 256
EPOCHS = 200
PATIENCE = 50
BATCH = 16

MODELS = [
    {"name": "yolov8l", "weights": "yolov8l.pt"},
    {"name": "yolo12l", "weights": "yolo12l.pt"},
    {"name": "rtdetr-l", "weights": "rtdetr-l.pt"},
]

torch.manual_seed(SEED)
exp_id = datetime.now().strftime("%Y%m%d_%H%M%S")
print(f"Experiment: {exp_id}")
print(f"Models: {[m['name'] for m in MODELS]}")

## Train Models

In [ ]:
results = []

for idx, model_cfg in enumerate(MODELS, 1):
    name = model_cfg['name']
    print(f"\n[{idx}/{len(MODELS)}] Training {name}")
    
    model = RTDETR(model_cfg['weights']) if 'rtdetr' in name.lower() else YOLO(model_cfg['weights'])
    
    model.train(
        data=str(YOLO_DIR / "config.yaml"),
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        patience=PATIENCE,
        batch=BATCH,
        seed=SEED,
        name=f"{exp_id}_{name}",
        project="runs",
        plots=True,
        verbose=False
    )
    
    val_metrics = model.val(split='val', verbose=False)
    test_metrics = model.val(split='test', verbose=False)
    
    results.append({
        'model': name,
        'val_precision': float(val_metrics.box.mp),
        'val_recall': float(val_metrics.box.mr),
        'val_map50': float(val_metrics.box.map50),
        'val_map': float(val_metrics.box.map),
        'test_precision': float(test_metrics.box.mp),
        'test_recall': float(test_metrics.box.mr),
        'test_map50': float(test_metrics.box.map50),
        'test_map': float(test_metrics.box.map),
    })
    
    print(f"{name}: test_map50={results[-1]['test_map50']:.4f}")

## Results

In [ ]:
df = pd.DataFrame(results)

summary = {
    'exp_id': exp_id,
    'seed': SEED,
    'epochs': EPOCHS,
    'img_size': IMG_SIZE,
    'batch': BATCH,
    'patience': PATIENCE,
    'models': [m['name'] for m in MODELS],
    'results': results,
    'best_model': results[df['test_map50'].idxmax()]['model'] if len(results) > 0 else None,
    'best_test_map50': float(df['test_map50'].max()) if len(results) > 0 else 0.0
}

with open(RESULTS_DIR / f"{exp_id}.json", 'w') as f:
    json.dump(summary, f, indent=2)

print("\nResults")
print(df[['model', 'val_map50', 'test_map50', 'test_precision', 'test_recall']].to_string(index=False))
print(f"\nBest: {summary['best_model']} (test_map50={summary['best_test_map50']:.4f})")
print(f"Saved: results/{exp_id}.json")

## Analysis

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

df.plot(x='model', y=['val_map50', 'test_map50'], kind='bar', ax=axes[0])
axes[0].set_title('mAP50 Comparison')
axes[0].set_ylabel('mAP50')
axes[0].legend(['Val', 'Test'])

df.plot(x='model', y=['test_precision', 'test_recall'], kind='bar', ax=axes[1])
axes[1].set_title('Test Metrics')
axes[1].set_ylabel('Score')
axes[1].legend(['Precision', 'Recall'])

plt.tight_layout()
plt.savefig(RESULTS_DIR / f"comparison_{exp_id}.png", dpi=150)
plt.show()

In [ ]:
model_name = MODELS[0]['name']
csv_file = Path(f"runs/{exp_id}_{model_name}/results.csv")

if csv_file.exists():
    training_df = pd.read_csv(csv_file)
    training_df.columns = training_df.columns.str.strip()
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    axes[0,0].plot(training_df['train/box_loss'], label='Train')
    axes[0,0].plot(training_df['val/box_loss'], label='Val')
    axes[0,0].set_title('Box Loss')
    axes[0,0].legend()
    axes[0,0].grid(alpha=0.3)
    
    axes[0,1].plot(training_df['metrics/precision(B)'])
    axes[0,1].set_title('Precision')
    axes[0,1].grid(alpha=0.3)
    
    axes[1,0].plot(training_df['metrics/recall(B)'])
    axes[1,0].set_title('Recall')
    axes[1,0].grid(alpha=0.3)
    
    axes[1,1].plot(training_df['metrics/mAP50(B)'])
    axes[1,1].set_title('mAP50')
    axes[1,1].grid(alpha=0.3)
    
    for ax in axes.flat:
        ax.set_xlabel('Epoch')
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / f"curves_{exp_id}_{model_name}.png", dpi=150)
    plt.show()